<a href="https://colab.research.google.com/github/madarahr/Big_Data/blob/main/Big_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu18

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
us_sports_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
us_sports_df.show()

In [ ]:
us_sports_df.printSchema()

In [ ]:
# Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [ ]:
# Create the list of struct fields
#schema = [StructField(StructField("star_rating", IntegerType(), True), StructField("helpful_votes", IntegerType(), True), StructField("total_votes", IntegerType(), True), StructField("vine", StringType(), True)]
schema = [StructField("marketplace", StringType(), True), StructField("customer_id", IntegerType(), True),
          StructField("review_id", StringType(), True), StructField("product_id", StringType(), True),
          StructField("product_parent", IntegerType(), True), StructField("product_title", StringType(), True),
          StructField("product_category", StringType(), True), StructField("star_rating", IntegerType(), True),
          StructField("helpful_votes", IntegerType(), True), StructField("total_votes", IntegerType(), True),
          StructField("vine", StringType(), True), StructField("verified_purchase", StringType(), True),
          StructField("review_headline", StringType(), True), StructField("review_body", StringType(), True), StructField("review_date", StringType(), True)]

In [ ]:
# Pass in our fields
final = StructType(fields=schema)

In [ ]:
# Read our data with our new schema
sports_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"), sep="\t", header=True, schema=final)
sports_df.printSchema()

In [ ]:
sports_df.count()

In [ ]:
# drops duplicate rows from dataframe
noDuplicate_df = sports_df.dropDuplicates()

In [ ]:
# Drop null values
sports_dropna_df = noDuplicate_df.dropna()

In [ ]:
sports_dropna_df.count()

In [ ]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

In [ ]:
# Filter for only columns with verified_purchase only
verified_df = sports_dropna_df.filter(col("verified_purchase") == "Y")

In [ ]:
# Filter
cleaned_df= verified_df.filter("helpful_votes>1")

In [ ]:
columns_to_drop = ["marketplace", "product_category", "verified_purchase", "review_head;ine", "review_body"]
lean_df = cleaned_df.drop(*columns_to_drop)

In [ ]:
# Filter
final_df= lean_df.filter("total_votes>1")

In [ ]:
from pyspark.sql.functions import col, countDistinct
customer_count = final_df.agg(countDistinct(col("customer_id")).alias("count"))

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Create user dataframe to match review_id table
review0_df = final_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review0_df.show(5)

In [ ]:
review0_df.printSchema()

In [ ]:
from pyspark.sql.functions import col, unix_timestamp, to_date

In [ ]:
review_id_df = review0_df.withColumn('review1_date', 
                   to_date(unix_timestamp(col('review_date'), 'yyyy-MM-dd').cast("timestamp")))

In [ ]:
review_id_df.show()

In [ ]:
review_id_df.printSchema()

In [ ]:
# Create user dataframe to match products table
products_df = final_df.select(["product_id", "product_title"])
products_df.show(5)

In [ ]:
# Create user dataframe with unique customer IDs
distinct_df = final_df.select(["customer_id"]).distinct()

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import monotonically_increasing_id,row_number

In [ ]:
customers_df =distinct_df.withColumn("customer_count",row_number().over(Window.orderBy(monotonically_increasing_id())))
customers_df.show(5)

In [ ]:
# Create user dataframe to match vine table
vine_df = final_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://challenge.cm4ewondqiwy.us-east-2.rds.amazonaws.com:5432/Challenge"
config = {"user":'postgres', 
          "password": "bootcamp1", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to review_id table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write dataframe to customers table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write dataframe to vine_table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [ ]:
# Write dataframe to products in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Filter dataframe with vine reviews
vineY_df = vine_df.filter(col("vine") == "Y")
vineY_df.count()

In [ ]:
# Filter dataframe with non-vine reviews
vineN_df = vine_df.filter(col("vine") == "N")
vineN_df.count()